# Full Analysis of Videos from Experiment on December 2, 2021 with 72 bar Saturation Pressure

Having explored the bubble growth data in `20211202_72bar_eda.ipynb` and improved the method for fitting the bubble growth model to data in `20211202_72bar_compare_fit_methods.ipynb`, I will now analyze the full dataset here.

In [2]:
# messes up unpickling
%load_ext autoreload
%autoreload 2

# sets plotting font to Arial
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

# standard libraries
import os
import glob
import pickle as pkl

# 3rd party libraries
import numpy as np
import matplotlib.pyplot as plt

# custom libraries
import sys
sys.path.append('../../libs/')
import fn
import flow
import plot.bubble as pltb
from conversions import *

# comes after imports from ../../libs/ because I unfortunately 
# created files with the same name there and in ../src/
sys.path.append('../src/')
import objproc as op
import bubble
import analytics as an

### PARAMETERS ###
# directory template containing data files
data_dir_tmp = '../input/ppg_co2/20211202_72bar/*'
# data filename
data_filename = 'data.pkl'
# data for polyol-CO2 mixture 
# make this on Wang group server with `bi_surf`--read README.txt and copy 1k3f_22c.csv
polyol_data_file = '../input/3k2f_31c.csv'
# equation of state data for CO2 at desired temperature 
eos_co2_file = '../input/eos_co2_25-0C.csv'

In [6]:
%%time
# about 1 minute per bubble

# GROWTH MODEL PARAMETERS
growth_fn = bubble.grow
dt = 1E-10 # [s]
R_nuc = 3E-9 # higher value bypasses numerical challenges at nm scale [m]
# make this on Wang group server with `bi_surf`--read README.txt and copy 1k3f_22c.csv
polyol_data_file = '../input/3k2f_31c.csv'
# equation of state data for CO2 at desired temperature 
eos_co2_file = '../input/eos_co2_25-0C.csv'

# FITTING PARAMETERS
frac_lo = 0.95
frac_hi = 1.05
# guess for effective diffusivity constant [m^2/s]
D_lo = 1E-9
D_hi = 8E-9
fit_fn_params = {'err_tol' : 0.005, 'err_fn' : an.calc_abs_sgn_mse}
exp_ratio_tol = 0.03

# saving and loading
save_path = '../output/20211202_72bar_21-32_35-74_debug.pkl'
save_freq = 3



# globs list of data directories
data_dir_list = glob.glob(data_dir_tmp)

#***BROKEN***

growth_data = an.fit_D_t_nuc(data_filename, data_dir_list, polyol_data_file, eos_co2_file,
                             frac_lo, frac_hi, D_lo, D_hi, growth_fn, dt, R_nuc, 
                             fit_fn_params, exp_ratio_tol, show_plots=False,
                            save_path=save_path, save_freq=save_freq)

36 R =  3.944717226683015 v =  0.8699552354223918  vs.  0.8600984947868408
37 R =  4.415613262359065 v =  0.845169983149204  vs.  0.8600984947868408
38 R =  3.3368847308126526 v =  0.8852897023809918  vs.  0.8600984947868408
40 R =  3.80592042319279 v =  0.9345880577840989  vs.  0.8600984947868408
41 R =  3.778462378939235 v =  0.8155191755585  vs.  0.8600984947868408
42 R =  3.536955250216245 v =  0.7356597813514928  vs.  0.8600984947868408
44 R =  4.449921480017429 v =  0.7986863252134222  vs.  0.8600984947868408
45 R =  3.6179830142995124 v =  0.6386831951345278  vs.  0.8600984947868408
50 R =  3.6179830142995124 v =  0.16624709139997484  vs.  0.8600984947868408
81 R =  3.83672878428012 v =  0.9140417428054907  vs.  0.8600984947868408
103 R =  3.904729526937109 v =  0.8332875614609467  vs.  0.8600984947868408
104 R =  4.016304854697718 v =  0.874627431883102  vs.  0.8600984947868408
109 R =  4.191478975130988 v =  0.5717320444675483  vs.  0.8600984947868408
130 R =  4.08742840935378

AssertionError: Nucleation time must be before times provided.

In [7]:
# loads data if already available--otherwise throws FileNotFoundError
dataset = an.fit_D_t_nuc_load(save_path, save_path, save_freq=3)

Could not load data from ../output/20211202_72bar_21-32_35-74_debug.pkl
dict_keys([])


TypeError: fit_D_t_nuc() missing 13 required positional arguments: 'data_filename', 'data_dir_list', 'polyol_data_file', 'eos_co2_file', 'frac_lo', 'frac_hi', 'D_lo', 'D_hi', 'growth_fn', 'dt', 'R_nuc', 'fit_fn_params', and 'exp_ratio_tol'

In [8]:
with open(save_path[:-4] + '_test.pkl', 'rb') as f:
    d = pkl.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../output/20211202_72bar_21-32_35-74_debug_test.pkl'

## $t^{1/2}$ Fit

In [11]:
# fits Epstein - Plesset model
save_path = '../output/20211202_72bar_sqrt.pkl'
# GROWTH MODEL PARAMETERS
growth_fn = bubble.sqrt_fit
dt = 1E-9 # [s]
R_nuc = 3E-9 # higher value bypasses numerical challenges at nm scale [m]

# FITTING PARAMETERS
frac_lo = 0.95
frac_hi = 1.05
# guess for effective diffusivity constant [m^2/s]
D_lo = 1E-9
D_hi = 1E-8
err_tol = 0.003
err_fn = an.calc_abs_sgn_mse
rms_tol = -0.02 # keep high--this is used to fit D, must allow for noise in data
exp_ratio_tol = 0.01
fit_fn_params = {}
save_freq = 1
fit_proc = an.scipy_param_fit

# PLOTTING PARAMETERS
# x_lim = [1E-2, 10]
# y_lim = [1, 30]

# USER PARAMETERS
n_fit = -1

# globs list of data directories
data_dir_list = glob.glob(data_dir_tmp)

# fits effective diffusivity D and nucleation time t_nuc
data = an.fit_D_t_nuc(data_filename, data_dir_list, polyol_data_file,
                        eos_co2_file, frac_lo, frac_hi,
                        D_lo, D_hi, growth_fn, dt, R_nuc, fit_fn_params,
                        exp_ratio_tol, n_fit=n_fit, show_plots=False,
                        save_path=save_path, save_freq=save_freq, fit_proc=fit_proc)

36 R =  3.944717226683015 v =  0.8699552354223918  vs.  0.8600984947868408
37 R =  4.415613262359065 v =  0.845169983149204  vs.  0.8600984947868408
38 R =  3.3368847308126526 v =  0.8852897023809918  vs.  0.8600984947868408
40 R =  3.80592042319279 v =  0.9345880577840989  vs.  0.8600984947868408
41 R =  3.778462378939235 v =  0.8155191755585  vs.  0.8600984947868408
42 R =  3.536955250216245 v =  0.7356597813514928  vs.  0.8600984947868408
44 R =  4.449921480017429 v =  0.7986863252134222  vs.  0.8600984947868408
45 R =  3.6179830142995124 v =  0.6386831951345278  vs.  0.8600984947868408
50 R =  3.6179830142995124 v =  0.16624709139997484  vs.  0.8600984947868408
81 R =  3.83672878428012 v =  0.9140417428054907  vs.  0.8600984947868408
103 R =  3.904729526937109 v =  0.8332875614609467  vs.  0.8600984947868408
104 R =  4.016304854697718 v =  0.874627431883102  vs.  0.8600984947868408
109 R =  4.191478975130988 v =  0.5717320444675483  vs.  0.8600984947868408
130 R =  4.08742840935378

0 R =  4.421628945155416 v =  0.7932080072934209  vs.  0.8614519197462353
6 R =  3.877837030746723 v =  1.9695649439627587  vs.  0.8614519197462353
10 R =  3.578825429458237 v =  0.7876635196141054  vs.  0.8614519197462353
11 R =  3.83672878428012 v =  0.8866766938892351  vs.  0.8614519197462353
21 R =  4.29587465532342 v =  -1.0083501359032807  vs.  0.8614519197462353
29 R =  3.536955250216245 v =  1.8598375377755438  vs.  0.8614519197462353
33 R =  3.536955250216245 v =  2.0182925636741844  vs.  0.8614519197462353
55 R =  3.536955250216245 v =  -0.1266431248503506  vs.  0.8614519197462353

Analyzing bubble 0 at 0.083 m.


Analyzed 46 bubbles.


Analyzing bubble 10 at 0.083 m.


Analyzed 47 bubbles.


Analyzing bubble 11 at 0.083 m.


Analyzed 48 bubbles.


Analyzed videos taken at distance 0.083 m.
There are 52 videos to analyze.

Saving after 48 bubbles analyzed.


Analyzed videos taken at distance 0.081 m.
There are 52 videos to analyze.

Saving after 48 bubbles analyzed.

15 R =  

1433 R =  4.29587465532342 v =  -0.4942390954552659  vs.  0.6935105807851352
1464 R =  3.3368847308126526 v =  0.7491766274403198  vs.  0.6935105807851352
1466 R =  13.742980149240719 v =  0.8623094791604046  vs.  0.6935105807851352
1557 R =  4.487215942936821 v =  0.4401549448977564  vs.  0.6935105807851352
1572 R =  7.256635272373031 v =  0.309039715868953  vs.  0.6935105807851352
1601 R =  3.3368847308126526 v =  2.0459440220727814  vs.  0.6935105807851352
1663 R =  4.523703784147474 v =  -0.4646868943660224  vs.  0.6935105807851352
1680 R =  3.8991581926678895 v =  -5.149379961549954  vs.  0.6935105807851352
1703 R =  4.349095305848652 v =  10.112977374229267  vs.  0.6935105807851352
1716 R =  5.121256305448452 v =  -6.517837586903038  vs.  0.6935105807851352
1723 R =  3.5257575864879414 v =  13.281881832868118  vs.  0.6935105807851352
1731 R =  5.24675050168675 v =  4.0056008688894424  vs.  0.6935105807851352
1735 R =  20.40953647511482 v =  0.4381148367693325  vs.  0.693510580785

3115 R =  20.675506008345486 v =  7.699922357009968  vs.  0.6935105807851352
3128 R =  4.591902926693419 v =  6.280676274750017  vs.  0.6935105807851352
3132 R =  20.38917803754699 v =  91.31617789642297  vs.  0.6935105807851352
3135 R =  4.868684477434043 v =  11.126981232562246  vs.  0.6935105807851352
3149 R =  13.313499659011624 v =  0.6862657859565322  vs.  0.6935105807851352
3170 R =  4.751548606504007 v =  0.7730033993940432  vs.  0.6935105807851352
3199 R =  3.732949234404223 v =  0.8017066104527834  vs.  0.6935105807851352
3236 R =  5.0191721382170575 v =  2.3071424410181636  vs.  0.6935105807851352
3241 R =  26.208925413184218 v =  15.953176267508091  vs.  0.6935105807851352
3244 R =  4.157925941383848 v =  0.24534767242905048  vs.  0.6935105807851352
3270 R =  3.5594192274884646 v =  1.1068098983619836  vs.  0.6935105807851352
3273 R =  26.07151488061713 v =  2.6216222947347045  vs.  0.6935105807851352
3280 R =  3.892045216803351 v =  10.014799924195135  vs.  0.6935105807851

../src\analytics.py:606: RankWarning: Polyfit may be poorly conditioned
  # bubble radius [m]


...

../src\analytics.py:606: RankWarning: Polyfit may be poorly conditioned
  # bubble radius [m]


3.6469459317746997  vs.  0.6918552611041401
822 R =  3.536955250216245 v =  0.6774944014751171  vs.  0.6918552611041401
831 R =  6.6672207601737945 v =  1.9920838227473951  vs.  0.6918552611041401
832 R =  4.644277201401288 v =  -1.508426961441745  vs.  0.6918552611041401
834 R =  5.769056105275061 v =  1.4829715631323432  vs.  0.6918552611041401
836 R =  5.0394986551552625 v =  9.621450391574049  vs.  0.6918552611041401
839 R =  4.746443582958427 v =  0.5350218066394241  vs.  0.6918552611041401
842 R =  4.449901888965327 v =  1.0549392767090155  vs.  0.6918552611041401
847 R =  4.186134419125851 v =  1.4428917243885697  vs.  0.6918552611041401
857 R =  5.865662855747546 v =  0.6617881294146536  vs.  0.6918552611041401
861 R =  3.7155536670544738 v =  1.579918474235027  vs.  0.6918552611041401
865 R =  7.856452330347953 v =  -0.8798173321452827  vs.  0.6918552611041401
868 R =  4.534119051391347 v =  0.038677821329684677  vs.  0.6918552611041401
870 R =  3.7155536670544738 v =  0.80580

744 R =  5.0636976540311425 v =  -0.3049535677448179  vs.  0.6968100183633289
751 R =  4.370061821081601 v =  -0.8693123815399943  vs.  0.6968100183633289
758 R =  10.414334188547603 v =  2.30564772890341  vs.  0.6968100183633289
785 R =  3.448735974492251 v =  1.524178720466854  vs.  0.6968100183633289
804 R =  3.7155536670544738 v =  1.0901757416697302  vs.  0.6968100183633289
824 R =  4.027183625165731 v =  0.42345136934561434  vs.  0.6968100183633289
829 R =  3.7155536670544738 v =  1.1914266306089467  vs.  0.6968100183633289
830 R =  3.536955250216245 v =  0.871819308708135  vs.  0.6968100183633289
864 R =  6.830569909772665 v =  -1.629981916492247  vs.  0.6968100183633289

Analyzing bubble 37 at 0.083 m.

Failed to fit data

Analyzed 75 bubbles.


Analyzing bubble 464 at 0.083 m.


Analyzed 76 bubbles.


Analyzing bubble 506 at 0.083 m.


Analyzed 77 bubbles.


Analyzed videos taken at distance 0.083 m.
There are 52 videos to analyze.

Saving after 77 bubbles analyzed.

0 R =  3.



Analyzing bubble 206 at 0.081 m.

Failed to fit data

Analyzed 95 bubbles.


Analyzed videos taken at distance 0.081 m.
There are 52 videos to analyze.

Saving after 95 bubbles analyzed.

0 R =  4.3196434612935 v =  0.6400387350801802  vs.  0.6951621573622259
4 R =  3.6179830142995124 v =  0.8630819146499034  vs.  0.6951621573622259
6 R =  3.80592042319279 v =  0.6676962378530791  vs.  0.6951621573622259
22 R =  3.6179830142995124 v =  -0.18507269516762406  vs.  0.6951621573622259
38 R =  4.165953693460643 v =  1.5397721164479743  vs.  0.6951621573622259
53 R =  3.536955250216245 v =  -1.5431083004687005  vs.  0.6951621573622259

Analyzing bubble 0 at 0.079 m.

Failed to fit data

Analyzed 96 bubbles.


Analyzing bubble 6 at 0.079 m.


Analyzed 97 bubbles.


Analyzed videos taken at distance 0.079 m.
There are 52 videos to analyze.

Saving after 97 bubbles analyzed.

10 R =  3.3368847308126526 v =  0.6531984674096406  vs.  0.6951621573622259
17 R =  3.83672878428012 v =  0.7597384635

0 R =  4.258320033015831 v =  0.7433644701250526  vs.  0.6968100183633289
18 R =  3.3368847308126526 v =  3.441691201889324  vs.  0.6968100183633289

Analyzing bubble 0 at 0.067 m.


Analyzed 119 bubbles.


Analyzed videos taken at distance 0.067 m.
There are 52 videos to analyze.

Saving after 119 bubbles analyzed.

3 R =  3.3368847308126526 v =  0.20910704184263496  vs.  0.6968100183633289

Analyzed videos taken at distance 0.067 m.
There are 52 videos to analyze.

Saving after 119 bubbles analyzed.

17 R =  7.6326743405629305 v =  0.6249527383341245  vs.  0.6968100183633289
20 R =  4.265908684769089 v =  0.2250247239881095  vs.  0.6968100183633289
21 R =  4.318401805030347 v =  -0.524686156521674  vs.  0.6968100183633289
22 R =  3.7155536670544738 v =  1.0524707592162026  vs.  0.6968100183633289
24 R =  3.536955250216245 v =  0.550773865457167  vs.  0.6968100183633289

Analyzing bubble 17 at 0.065 m.


Analyzed 120 bubbles.


Analyzed videos taken at distance 0.065 m.
There are 52 v